## Exploring the UK's coffee, tea and mate imports and exports of 2014

by Benjamin Woodling, 08 December 2015
based on a template from Tony Hirst and Michel Wermelinger, 19 November 2015

which was the project notebook for Week 4 of The Open University's [_Learn to code for Data Analysis_](http://futurelearn.com/courses/learn-to-code) course.

A country's economy depends, sometimes heavily, on its exports and imports. The United Nations Comtrade database provides data on global trade. It will be used to analyse the UK's imports and exports of coffee, tea and mate in 2014:

- How much does the UK export and import and is the balance positive (more exports than imports)? 
- Which are the main trading partners, i.e. from/to which countries does the UK import/export the most?
- Which are the regular customers, i.e. which countries buy coffee and tea from the UK every month?
- Which countries does the UK both import from and export to?
- For which countries does the UK have a higher value of export trade than import trade?

In [101]:
from pandas import *

## Getting and preparing the data

The data is obtained from the [United Nations Comtrade](http://comtrade.un.org/data/) website, by selecting the following configuration:

- Frequency: Monthly 
- Periods: All of 2014
- Reporter: United Kingdom
- Partners: All
- Flows: Imports and Exports
- HS (as reported) commodity codes: 09 (Coffee, tea, mate and spices), 0901 (Coffee, coffee husks and skins and coffee substitutes), 0902 (Tea), 0903 (Mate).

Clicking on 'Preview' results in a message that the data exceeds 500 rows and reveals a 'View API call' link at the bottom of the page. Clicking on it shows the URL, which has to be changed in two ways: `max=500` is increased to `max=5000` to make sure all data is loaded, and `&fmt=csv` is added at the end to obtain the data in CSV format. 

In addition, the commodity code has to be read as a string, to not lose the leading zero.

In [185]:
URL = 'http://comtrade.un.org/api/get?max=5000&type=C&freq=M&px=HS&ps=2014&r=826&p=all&rg=1%2C2&cc=0901%2C0902%2C0903&fmt=csv'
coffee = read_csv(URL, dtype={'Commodity Code':str})

The data only covers the first nine months. Most columns are irrelevant for this analysis, or contain always the same value, like the year and reporter columns. The commodity code is transformed into a short but descriptive text and only the relevant columns are selected.

In [186]:
def coffeeType(code):
    if code == '0901': # coffee
        return 'coffee'
    if code == '0902': # tea
        return 'tea'
    if code == '0903': # mate
        return 'mate' 
    return 'unknown'

COMMODITY = 'Coffee, tea and mate'
coffee[COMMODITY] = coffee['Commodity Code'].apply(coffeeType)
MONTH = 'Period'
PARTNER = 'Partner'
FLOW = 'Trade Flow'
VALUE = 'Trade Value (US$)'
headings = [MONTH, PARTNER, FLOW, COMMODITY, VALUE]
coffee = coffee[headings]
coffee.head()

,Period,Partner,Trade Flow,"Coffee, tea and mate",Trade Value (US$)
0,201401,World,Imports,coffee,57703333
1,201401,World,Exports,coffee,28115190
2,201401,Afghanistan,Exports,coffee,8777
3,201401,Argentina,Exports,coffee,353172
4,201401,Australia,Exports,coffee,566986


The data contains the total imports and exports per month, under the 'World' partner. Those rows are removed to keep only the per-country data.

In [187]:
coffee = coffee[coffee[PARTNER] != 'World']
coffee.head()

,Period,Partner,Trade Flow,"Coffee, tea and mate",Trade Value (US$)
2,201401,Afghanistan,Exports,coffee,8777
3,201401,Argentina,Exports,coffee,353172
4,201401,Australia,Exports,coffee,566986
5,201401,Austria,Exports,coffee,260268
6,201401,Belgium,Imports,coffee,1776588


## Total trade flow

To answer the first question, 'how much does the UK export and import and is the balance positive (more exports than imports)?',
the dataframe is split into two groups: exports from the UK and imports into the UK. The  trade values within each group are summed up to get the total trading.

In [188]:
grouped = coffee.groupby([FLOW])
grouped[VALUE].aggregate(sum)

Trade Flow
Exports     537051414
Imports    1207612154
Name: Trade Value (US$), dtype: int64

This shows a trade deficit of about 670 million dollars. Coffee is an expensive substance. Or is it? How much is the trade value of coffee compared to tea and mate? 

In [189]:
grouped = coffee.groupby([FLOW,COMMODITY])
grouped[VALUE].aggregate(sum)

Trade Flow  Coffee, tea and mate
Exports     coffee                  383484099
            mate                       175075
            tea                     153392240
Imports     coffee                  833369206
            mate                      1087559
            tea                     373155389
Name: Trade Value (US$), dtype: int64

Coffee accounts for between two and three times the trade value of tea while mate is very minimal in comparison to both with less than 0.1% of the total trade value.

## Main trade partners

To address the second question, 'Which are the main trading partners, i.e. from/to which countries does the UK import/export the most?', the dataframe is split by country instead, and then each group aggregated for the total trade value. This is done separately for all imports and exports. Sources of coffee import and tea import to the UK are also listed. The results are sorted in descending order so that the main partners are at the top.

In [191]:
imports = coffee[(coffee[FLOW] == 'Imports')]
grouped = imports.groupby([PARTNER])
print('The UK imports coffee, mate and tea from', len(grouped), 'countries.')
print('The 5 biggest exporters to the UK are:')
grouped[VALUE].aggregate(sum).sort(inplace=False,ascending=False).head()

The UK imports coffee, mate and tea from 99 countries.
The 5 biggest exporters to the UK are:


Partner
Brazil      166582363
Kenya       129897199
Colombia     77800527
Germany      71872696
France       69506110
Name: Trade Value (US$), dtype: int64

In [223]:
exports = coffee[(coffee[FLOW] == 'Exports')]
grouped = exports.groupby([PARTNER])
print('The UK exports coffee, mate and tea to', len(grouped), 'countries.')
print('The 5 biggest importers from the UK are:')
grouped[VALUE].aggregate(sum).sort(inplace=False,ascending=False).head()

The UK exports coffee, mate and tea to 115 countries.
The 5 biggest importers from the UK are:


Partner
Ireland    93109602
France     61158069
Germany    59860765
Japan      28632167
Canada     26950996
Name: Trade Value (US$), dtype: int64

In [193]:
imports = coffee[(coffee[FLOW] == 'Imports') & (coffee[COMMODITY] == 'coffee')]
grouped = imports.groupby([PARTNER])
print('The UK imports coffee from', len(grouped), 'countries.')
print('The 5 biggest exporters to the UK are:')
grouped[VALUE].aggregate(sum).sort(inplace=False,ascending=False).head()

The UK imports coffee from 82 countries.
The 5 biggest exporters to the UK are:


Partner
Brazil      166403158
Colombia     77726395
Italy        68688888
France       68457027
Viet Nam     63042353
Name: Trade Value (US$), dtype: int64

In [194]:
imports = coffee[(coffee[FLOW] == 'Imports') & (coffee[COMMODITY] == 'tea')]
grouped = imports.groupby([PARTNER])
print('The UK imports tea from', len(grouped), 'countries.')
print('The 5 biggest exporters to the UK are:')
grouped[VALUE].aggregate(sum).sort(inplace=False,ascending=False).head()

The UK imports tea from 74 countries.
The 5 biggest exporters to the UK are:


Partner
Kenya        120422946
India         59371536
Singapore     49947379
Poland        19036047
Malawi        15914474
Name: Trade Value (US$), dtype: int64

## Regular importers

Given that there are two main commodities here, the third question, 'Which are the regular customers, i.e. which countries buy coffee and tea from the UK every month?', is meant in the sense that a regular customer imports both commodities every month.

The exports dataframe must first be limited to the coffee and tea commodities. Then if the exports dataframe is grouped by country, each country that fulfils these conditions will have exactly 24 rows (two commodities bought each of the twelve months). To see the countries, only the first month of one commodity has to be listed, as by definition it's the same countries every month and for the other commodity.

In [195]:
def buysEveryMonth(group):
    return len(group) == 24

exports = exports[(exports['Coffee, tea and mate']=='coffee') | (exports['Coffee, tea and mate']=='tea')]
grouped = exports.groupby([PARTNER])
regular = grouped.filter(buysEveryMonth)
regular[(regular[MONTH] == 201401) & (regular['Coffee, tea and mate'] == 'coffee')]

,Period,Partner,Trade Flow,"Coffee, tea and mate",Trade Value (US$)
4,201401,Australia,Exports,coffee,566986
5,201401,Austria,Exports,coffee,260268
7,201401,Belgium,Exports,coffee,652633
11,201401,Bulgaria,Exports,coffee,96362
13,201401,Canada,Exports,coffee,131984
17,201401,China,Exports,coffee,307362
24,201401,Cyprus,Exports,coffee,41342
26,201401,Czech Rep.,Exports,coffee,490058
28,201401,Denmark,Exports,coffee,469835
30,201401,Estonia,Exports,coffee,484


In [225]:
regular[VALUE].sum() / exports[VALUE].sum()

0.8948057755230117

Over 89% of the total UK exports are due to these regular customers.

In [229]:
print("The UK exports coffee, tea and mate products to", round(len(regular)/24), "countries all year round.")

The UK exports coffee, tea and mate products to 41 countries all year round.


## Bi-directional trade

To address the fourth question, 
'Which countries does the UK both import from and export to?', a pivot table is used to list the total export and import value for each country. The original dataframe 'coffee' showing all commodities of coffee, tea and mate is applicable here.   

In [204]:
countries = pivot_table(coffee, index=[PARTNER], columns=[FLOW], 
                        values=VALUE, aggfunc=sum)
countries.head()

Trade Flow,Exports,Imports
Partner,,
Afghanistan,8777,NaN
Algeria,62078,NaN
Angola,13520,NaN
Argentina,3440076,1501523
Australia,12386096,340570


Removing the rows with a missing value will result in only those countries with bi-directional trade flow with the UK.

In [198]:
countries.dropna()

Trade Flow,Exports,Imports
Partner,,
Argentina,3440076,1501523
Australia,12386096,340570
Austria,5789748,33949
Belgium,16278657,37679080
Bolivia,28322,687754
Brazil,12760350,166582363
Brunei Darussalam,13995,36251
Bulgaria,1400663,56195
Canada,26950996,1468424


There are 76 countries that both import and export coffee, mate or tea commodities with the UK.

## Trade Surplus Countries
For which countries does the UK have a higher value of export trade than import trade? 

In [219]:
countries = pivot_table(coffee, index=[PARTNER], columns=[FLOW], 
                        values=VALUE, aggfunc=sum)
countries = countries.fillna(0)
higher_export = countries[countries['Exports'] > countries['Imports']]
higher_export.head()

Trade Flow,Exports,Imports
Partner,,
Afghanistan,8777,0
Algeria,62078,0
Angola,13520,0
Argentina,3440076,1501523
Australia,12386096,340570


In [218]:
print('There are', len(higher_export), 'countries where the UK exports a higher trade value of coffee, tea and mate than it imports.') 

There are 81 countries where the UK exports a higher trade value of coffee, tea and mate than it imports.


In [221]:
higher_import = countries[countries['Imports'] > countries['Exports']]
higher_import.head()

Trade Flow,Exports,Imports
Partner,,
Bangladesh,0,23989
Belgium,16278657,37679080
Bolivia,28322,687754
Bouvet Island,0,1068037
Brazil,12760350,166582363


In [222]:
print('There are', len(higher_import), 'countries where the UK imports a higher trade value of coffee, tea and mate than it exports.') 

There are 57 countries where the UK imports a higher trade value of coffee, tea and mate than it exports.


## Conclusions

The coffee, tea and mate trade of the UK from January to December 2014 was analysed in terms of which countries the UK mostly depends on for income (exports) and goods (imports). Over the period, the UK had a trade deficit of about 670 million US dollars.

Total trade in coffee amounts to about 1.2 billion dollars while total trade in tea is about 0.5 billion dollars. Mate represents about 0.1% of the total market and need not be considered further. 

It is interesting to learn that the UK exports coffee and tea to more countries than those it imports from. This despite running a significant trade deficit overall. In value terms, Brazil is the largest exporter of coffee to the UK while Kenya exports the most tea.

There is considerable import and export also between the UK and European countries in these products. France, Germany and Italy rank highly on both import and export lists while Ireland is the UK's most profitable export market.

Regular trade is common with the UK exporting to 41 countries in every month of the year. The UK is heavily dependent on these regular customers who contribute close to 90% of the total export value.